# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sierra grande,-41.6060,-65.3557,293.05,35,43,4.40,AR,1711575628
1,1,blackmans bay,-43.0167,147.3167,282.89,82,8,1.47,AU,1711575628
2,2,longyearbyen,78.2186,15.6401,269.06,63,75,5.14,SJ,1711575628
3,3,arraial do cabo,-22.9661,-42.0278,297.13,88,75,2.57,BR,1711575629
4,4,edinburgh of the seven seas,-37.0676,-12.3116,292.50,93,100,11.18,SH,1711575629


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_by_city = city_data_df.hvplot.points(
    x = "Lng",
    y = "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    hover_cols = ["City"]
)

# Display the map
humidity_by_city

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_condition_df = city_data_df.loc[
    (city_data_df['Max Temp'] > 294.15) &
    (city_data_df['Max Temp'] < 300.15) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]
#My api key spat out the number in kelvin. I converted the boundaries to kelvin in order to do this project.


# Drop any rows with null values
ideal_weather_condition_df = ideal_weather_condition_df.dropna()

# Display sample data
ideal_weather_condition_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
103,103,al jumum,21.6169,39.6981,299.06,40,0,1.43,SA,1711575643
123,123,ascension,30.2169,-90.9001,294.82,50,0,3.60,US,1711575528
126,126,jashpurnagar,22.9000,84.1500,294.93,29,0,1.95,IN,1711575646
136,136,toliara,-23.3500,43.6667,297.81,74,0,3.12,MG,1711575647
424,424,arafat,21.3594,39.9664,298.80,45,0,0.70,SA,1711575692
425,425,mar del plata,-38.0023,-57.5575,297.56,71,0,2.24,AR,1711575666
457,457,taoudenni,22.6783,-3.9836,297.04,13,0,2.55,ML,1711575697
505,505,phalaborwa,-23.9430,31.1411,295.75,78,0,2.99,ZA,1711575703
535,535,pathein,16.7833,94.7333,295.57,86,0,3.70,MM,1711575635


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_condition_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = None

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
103,al jumum,SA,21.6169,39.6981,40,None
123,ascension,US,30.2169,-90.9001,50,None
126,jashpurnagar,IN,22.9000,84.1500,29,None
136,toliara,MG,-23.3500,43.6667,74,None
424,arafat,SA,21.3594,39.9664,45,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, 'Lat']
    longitude = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
al jumum - nearest hotel: No hotel found
ascension - nearest hotel: Cajun Country Inn
jashpurnagar - nearest hotel: No hotel found
toliara - nearest hotel: Ambary
arafat - nearest hotel: ابراج فجر الايمان
mar del plata - nearest hotel: Gran hotel Mónaco
taoudenni - nearest hotel: No hotel found
phalaborwa - nearest hotel: Hans Merensky Hotel & Estate
pathein - nearest hotel: Taan Taan Ta


,City,Country,Lat,Lng,Humidity,Hotel Name
103,al jumum,SA,21.6169,39.6981,40,No hotel found
123,ascension,US,30.2169,-90.9001,50,Cajun Country Inn
126,jashpurnagar,IN,22.9000,84.1500,29,No hotel found
136,toliara,MG,-23.3500,43.6667,74,Ambary
424,arafat,SA,21.3594,39.9664,45,ابراج فجر الايمان
425,mar del plata,AR,-38.0023,-57.5575,71,Gran hotel Mónaco
457,taoudenni,ML,22.6783,-3.9836,13,No hotel found
505,phalaborwa,ZA,-23.9430,31.1411,78,Hans Merensky Hotel & Estate
535,pathein,MM,16.7833,94.7333,86,Taan Taan Ta


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
humidity_by_city = city_data_df.hvplot.points(
    x = "Lng",
    y = "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "Humidity",
    hover_cols = ["City", "Humidity"]
)

# Display the map
humidity_by_city

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)